In [2]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

# Summary

This notebook creates an expanded wildland urban interface (WUI) data layer that establishes the bounds of the EEMS modeling area of interest. This is done by combining multiple WUI datasets, as well as community wildfire protection plan (CWPP) administrative zones. We then clip out the ocean. These various data layers were aquired from Cal Fire, Santa Barbara County Fire, SB City Fire, Los Padres National Forest, as well as digitized from various CWPPs.

**Input Data Layers:**
- [Multiagency WUIs](https://databasin.org/datasets/492eb38ab0374de99d15d6c6a132e337/)
- [CWPP Boundaries](https://databasin.org/datasets/54b02e8e341149f7a09aa6507cfebdd7/)
- [SBC Boundary](https://databasin.org/datasets/1cdc07adea7d4dee9cc1f07ab44cfef3/)

In [3]:
arcpy.ResetEnvironments()
gdb_name = "Expanded_WUI.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

**Load the various data layers used to create the expanded WUI**

In [4]:
wuis = ["Carpinteria_Summerland_WUI_Boundary",
       "Goleta_WUI_Boundary",
       "Montecito_WUI_Boundary",
       "Santa_Barbara_City_WUI_Boundary",
       "WUI_Interface",
       "WUI_Intermix"]

cwpps = ["Carpinteria_Summerland_CWPP_Boundary",
        "Gaviota_Coast_CWPP_Boundary",
        "Mission_Canyon_CWPP_Boundary",
        "Montecito_CWPP_Boundary",
        "San_Marcos_Pass_CWPP_Boundary",
        "Tepusquet_CWPP_Boundary",
        "SBC_CWPP", "GoletaCWPPBoundary"]

crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')

for sf in wuis:
    path = os.path.join(THIS_FOLDER, "in_data/{}.shp".format(sf))
    arcpy.CopyFeatures_management(path, sf)
    arcpy.Project_management(sf, "{}_rpj".format(sf), crs)

for sf in cwpps:
    path = os.path.join(THIS_FOLDER, "in_data/{}.shp".format(sf))
    arcpy.CopyFeatures_management(path, sf)
    arcpy.Project_management(sf, "{}_rpj".format(sf), crs)

path = os.path.join(THIS_FOLDER, "in_data/SBC_Boundary.shp")
arcpy.CopyFeatures_management(path, "SBC_Boundary")
arcpy.Project_management("SBC_Boundary", "SBC_Boundary_rpj", crs)

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Expanded_WUI.gdb\\SBC_Boundary_rpj'>

**Merge the WUI Intermix and Interface layers and then buffer them by 1.5 miles (the distance used by Cal Fire to create their WUI influence zone)**

In [5]:
arcpy.Merge_management(["WUI_Interface_rpj", "WUI_Intermix_rpj"], "WUI_Merge")
arcpy.analysis.Buffer("WUI_Merge", "WUI_Merge_Buf", "1.5 Miles")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Expanded_WUI.gdb\\WUI_Merge_Buf'>

**Merge the previous merged and buffered layer with the various organizational WUIs and CWPPs**

In [6]:
cwpp_wui = ["WUI_Merge_Buf"]

for sf in wuis:
    cwpp_wui.append("{}_rpj".format(sf))
    
for sf in cwpps:
    cwpp_wui.append("{}_rpj".format(sf))

arcpy.Merge_management(cwpp_wui, "Expanded_WUI_wOcean")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Expanded_WUI.gdb\\Expanded_WUI_wOcean'>

**Remove the portions that are in the ocean, aggregate the result to be one geometric feature by using Aggregate Polygon with a 500 ft radius, then export the resulting layer to the out_data folder**

In [7]:
dir_path = os.path.join(THIS_FOLDER, "out_data/expanded_wui")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.Clip_analysis("Expanded_WUI_wOcean", "SBC_Boundary_rpj", 
                    "Expanded_WUI_nonAgr")
arcpy.AggregatePolygons_cartography("Expanded_WUI_nonAgr", 
                                    "Expanded_WUI", "500 Feet")

arcpy.conversion.FeatureClassToShapefile("Expanded_WUI", os.path.join(THIS_FOLDER, "out_data/expanded_wui"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\expanded_wui'>